In [ ]:
!pip install pdfplumber
!pip install openpyxl
!pip install camelot-py[cv]
!pip install tabula-py
!pip install pdfplumber
!pip install pandas
!pip install pytesseract 
!pip install pdf2image


In [ ]:
import pdfplumber
import fitz # PyMuPDF
import pandas as pd
import os

# Directorios de entrada y salida
pdf_directory = "pdf"  # Ruta de los archivos PDF
output_directory = "excel"  # Carpeta para guardar los archivos Excel
os.makedirs(output_directory, exist_ok=True)

# Iterar sobre los archivos PDF en el directorio
for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        filepath = os.path.join(pdf_directory, filename)
        output_path = os.path.join(output_directory, filename.replace(".pdf", ".xlsx"))
        
        print(f"Procesando archivo: {filename}")
        
        try:
            # Abrir el PDF y extraer tablas
            with pdfplumber.open(filepath) as pdf:
                all_tables = []
                for page in pdf.pages:
                    tables = page.extract_tables()
                    for table in tables:
                        df = pd.DataFrame(table)
                        all_tables.append(df)
                
                # Extraer texto con PyMuPDF
                doc = fitz.open(filepath)
                all_text = ""
                for page_num in range(len(doc)):
                    page = doc.load_page(page_num)
                    all_text += page.get_text()
                
                # Guardar en Excel si se encontraron tablas
                if all_tables:
                    with pd.ExcelWriter(output_path) as writer:
                        for i, df in enumerate(all_tables):
                            df.to_excel(writer, sheet_name=f"Tabla_{i+1}", index=False, header=False)
                        # Guardar el texto en una hoja separada
                        text_df = pd.DataFrame([all_text.split('\n')])
                        text_df.to_excel(writer, sheet_name="Texto", index=False, header=False)
                    print(f"Archivo guardado en: {output_path}")
                else:
                    print(f"No se encontraron tablas en {filename}")
        
        except Exception as e:
            print(f"Error al procesar {filename}: {e}")

ModuleNotFoundError: No module named 'PyMuPDF'